# Churn Portfolio Project: Data Cleaning and Initial Exploration
## by Josh Murray
*https://github.com/jcmurray89/portfolioChurn.git*

## Initial Setup and Data Exploration

**Scenario**
This is reviewing a dataset from Kaggle showing customer accounts at an e-commerce site, several stats related to the account for the previous month's activity, and whether or not they have cancelled in the past month, aka, "churned". 

"Churn" or not is a boolean target we will analyze. 

**Data:**

From: 
https://www.kaggle.com/datasets/samuelsemaya/e-commerce-customer-churn

by Samuel Semaya · Updated 22 days "go as of 8/28,/ therefore on 8/6/24

Features
- Tenure: Tenure of a customer in the company (numeric)
- WarehouseToHome: Distance between the warehouse to the customer's home (numeric)
- NumberOfDeviceRegistered: Total number of devices registered to a particular customer (numeric)
- PreferedOrderCat: Preferred order category of a customer in the last month (categorical)
- SatisfactionScore: Satisfactory score of a customer on service (numeric)
- MaritalStatus: Marital status of a customer (categorical)
- NumberOfAddress: Total number of addresses added for a particular customer (numeric)
- Complaint: Whether any complaint has been raised in the last month (binary)
- DaySinceLastOrder: Days since last order by customer (numeric)
- CashbackAmount: Average cashback in last month (n:meric)
- Churn: Churn flag target vriable, binary)


### Setup

Import necessary libraries, read in the data as a pandas dataframe, and review the first few rows. 

In [48]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [50]:
rough_churn = pd.read_csv("C:/Users/joshc/Desktop/CPU Engineer Stuff/Project Datasets/Churn example/CHURN_archive/data_ecommerce_customer_churn.csv")
rough_churn.head()

,Tenure,WarehouseToHome,NumberOfDeviceRegistered,PreferedOrderCat,SatisfactionScore,MaritalStatus,NumberOfAddress,Complain,DaySinceLastOrder,CashbackAmount,Churn
0,15.0,29.0,4,Laptop & Accessory,3,Single,2,0,7.0,143.32,0
1,7.0,25.0,4,Mobile,1,Married,2,0,7.0,129.29,0
2,27.0,13.0,3,Laptop & Accessory,1,Married,5,0,7.0,168.54,0
3,20.0,25.0,4,Fashion,3,Divorced,7,0,NaN,230.27,0
4,30.0,15.0,4,Others,4,Single,8,0,8.0,322.17,0


**Questions:**

Based on the data above, here are some proposed initial questions for analysis: 

1. What is the average and median tenure by churn status?
2. What is the average and median satisfaction score by churn status? 
3. What is the average and median cashback amount by churn status?
4. What is the average and median distance from the warehouse by churn status?
5. What are the average and median days since last order by churn status?
6. What is the % churn based on marital status?
7. What is the % churn based on preferred order cat

Questions 1 - 5 look at numeric data, so we can look at summary statistics. 6-7 relate to categorical data, so we will look at the % churn based on each response. Technically, "Satisfaction score" is categorical, as the relative value between 1 and 2 may be different from that between 4 and 5, so it could also be analyzed as a bucket. 
egory?

For simplicity, we have chosen to ignore these factors, so we are removing two columns from the dataframe: 

In [52]:
drop_columns = ['NumberOfDeviceRegistered', 'NumberOfAddress']
rough_churn = rough_churn.drop(labels = drop_columns, axis = 1)
print(rough_churn.head(1))

   Tenure  WarehouseToHome    PreferedOrderCat  SatisfactionScore  \
0    15.0             29.0  Laptop & Accessory                  3   

  MaritalStatus  Complain  DaySinceLastOrder  CashbackAmount  Churn  
0        Single         0                7.0          143.32      0  


**Fixing Data Types**

pd.read_csv() tries to parse the data type, but it is not perfect, so it is important to double-check and make corrections as necessary. The next three blocks are the check before, the change, and the verification after. 

In [12]:
print(rough_churn.dtypes)

Tenure               float64
WarehouseToHome      float64
PreferedOrderCat      object
SatisfactionScore      int64
MaritalStatus         object
Complain               int64
DaySinceLastOrder    float64
CashbackAmount       float64
Churn                  int64
dtype: object


In [56]:
rough_churn['SatisfactionScore'] = rough_churn['SatisfactionScore'].astype('category')
rough_churn['Complain'] = rough_churn['Complain'].astype('boolean')
rough_churn['Churn'] = rough_churn['Churn'].astype('boolean')

In [16]:
print(rough_churn.dtypes)

Tenure               float64
WarehouseToHome      float64
PreferedOrderCat      object
SatisfactionScore      int64
MaritalStatus         object
Complain             boolean
DaySinceLastOrder    float64
CashbackAmount       float64
Churn                boolean
dtype: object


**Clarifying Column Names**

I found some of our column names to be less descriptive than needed, so I have added clarity. Others were too long, I have shortened their labels. Because this is free open-source test data, I have assigned hypothetical units. The project on Kaggle did not specify units. Were this a project in a work environment, we would need to work with subject-matter experts to determine the correct units of tenure and distance to the warehouse, etc. 

In [58]:
new_columns = {
    'Tenure': 'ActAgeMonths',
	'WarehouseToHome': 'KmToWarehouse',
	'NumberOfDeviceRegistered': 'DeviceCount',
	'Complain': 'Complaint30Days',
	'DaySinceLastOrder': 'DaysFromOrder',
	'SatisfactionScore': 'SatScore'
}
churn = rough_churn.rename(mapper = new_columns, axis = 1)
print(churn.head(1))

   ActAgeMonths  KmToWarehouse    PreferedOrderCat SatScore MaritalStatus  \
0          15.0           29.0  Laptop & Accessory        3        Single   

   Complaint30Days  DaysFromOrder  CashbackAmount  Churn  
0            False            7.0          143.32  False  


**Other Initial Exploration Methods**

pd.describe() gives summary values for numerical data. pd.info() gives insight about our values. The non-null counts and data types will be important as we go forward. 

In [20]:
print(churn.describe())

       ActAgeMonths  KmToWarehouse     SatScore  DaysFromOrder  CashbackAmount
count   3747.000000    3772.000000  3941.000000    3728.000000     3941.000000
mean      10.081398      15.650583     3.088302       4.531652      176.707419
std        8.498864       8.452301     1.381832       3.667648       48.791784
min        0.000000       5.000000     1.000000       0.000000        0.000000
25%        2.000000       9.000000     2.000000       2.000000      145.700000
50%        9.000000      14.000000     3.000000       3.000000      163.340000
75%       16.000000      21.000000     4.000000       7.000000      195.250000
max       61.000000     127.000000     5.000000      46.000000      324.990000


In [22]:
print(churn.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3941 entries, 0 to 3940
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ActAgeMonths      3747 non-null   float64
 1   KmToWarehouse     3772 non-null   float64
 2   PreferedOrderCat  3941 non-null   object 
 3   SatScore          3941 non-null   int64  
 4   MaritalStatus     3941 non-null   object 
 5   Complaint30Days   3941 non-null   boolean
 6   DaysFromOrder     3728 non-null   float64
 7   CashbackAmount    3941 non-null   float64
 8   Churn             3941 non-null   boolean
dtypes: boolean(2), float64(4), int64(1), object(2)
memory usage: 231.0+ KB
None


The "shape" method displays the (rows, columns) in a dataframe. 

In [184]:
print(churn.shape)

(3941, 10)


Finally, creating an index column: 

In [60]:
churn = churn.reset_index()
print(churn.head(3))

   index  ActAgeMonths  KmToWarehouse    PreferedOrderCat SatScore  \
0      0          15.0           29.0  Laptop & Accessory        3   
1      1           7.0           25.0              Mobile        1   
2      2          27.0           13.0  Laptop & Accessory        1   

  MaritalStatus  Complaint30Days  DaysFromOrder  CashbackAmount  Churn  
0        Single            False            7.0          143.32  False  
1       Married            False            7.0          129.29  False  
2       Married            False            7.0          168.54  False  


### Analysis

**Numeric Data Questions:**

Question 1: What is the average and median tenure by churn status?

In [62]:
# Select just the columns we need, then group, then aggregate: 
tenure = churn[['ActAgeMonths','Churn']]
tenure_stats = tenure.groupby('Churn').agg({'ActAgeMonths':['mean','median', 'std']})
print(tenure_stats)

      ActAgeMonths                 
              mean median       std
Churn                              
False    11.427160   10.0  8.347744
True      3.160656    1.0  5.306107


2. What is the average and median satisfaction score by churn status?
3. What is the average and median cashback amount by churn status?
4. What is the average and median distance from the warehouse by churn status?
5. What are the average and median days since last order by churn status?

Rinse and repeat for the next several questions where we will treat "Churn" True/False as a category and analyze that compared to numerical data. We will move through them with a looped funct. 

We will use mean, median, and standard deviation for a simple summary. urn

In [64]:
columns = ['SatScore','CashbackAmount','KmToWarehouse', 'DaysFromOrder'] # numeric columns
summary_methods = ['mean','median', 'std'] #chosen summary methods for the .agg() method
numeric_df_list = [] #empty list to populate

def numeric_col_analysis(column, sum_list):
    temp_df = churn[[column,'Churn']]
    temp_df[column] = temp_df[column].astype('float64') #guarantee the summary stats can be performed
    temp_df.dropna(inplace=True) # Remove null results from the temp df to maintain accurate results
    temp_df_stats = temp_df.groupby('Churn').agg({column:sum_list})
    return temp_df_stats

for col in columns: 
    numeric_df_list.append(numeric_col_analysis(col, summary_methods))

for df in numeric_df_list: print(df, "\n")

       SatScore                 
           mean median       std
Churn                           
False  3.020814    3.0  1.379647
True   3.415430    3.0  1.346218 

      CashbackAmount                   
                mean  median        std
Churn                                  
False     180.286847  166.08  50.019129
True      159.357285  149.34  37.841377 

      KmToWarehouse                 
               mean median       std
Churn                               
False     15.376187   13.0  8.381713
True      17.061889   15.0  8.677252 

      DaysFromOrder                 
               mean median       std
Churn                               
False      4.797996    4.0  3.645314
True       3.231861    2.0  3.497441 



We want to save each of these dataframes to their own variable: 

In [30]:
satscore_df = numeric_df_list[0]
cashback_df = numeric_df_list[1]
distance_df = numeric_df_list[2]
orderdays_df = numeric_df_list[3]

Verifying success for the above: 

In [35]:
print(satscore_df, "\n")
print(cashback_df, "\n")
print(distance_df, "\n")
print(orderdays_df, "\n")
print(tenure_stats, "\n")

       SatScore                 
           mean median       std
Churn                           
False  3.020814    3.0  1.379647
True   3.415430    3.0  1.346218 

      CashbackAmount                   
                mean  median        std
Churn                                  
False     180.286847  166.08  50.019129
True      159.357285  149.34  37.841377 

      KmToWarehouse                 
               mean median       std
Churn                               
False     15.376187   13.0  8.381713
True      17.061889   15.0  8.677252 

      DaysFromOrder                 
               mean median       std
Churn                               
False      4.797996    4.0  3.645314
True       3.231861    2.0  3.497441 

      ActAgeMonths                 
              mean median       std
Churn                              
False    11.427160   10.0  8.347744
True      3.160656    1.0  5.306107 



In [126]:
#How to access multi-level variables:
print(satscore_df.columns)
sat_true_mean = satscore_df[('SatScore',   'mean')][1]
print(sat_true_mean)

MultiIndex([('SatScore',   'mean'),
            ('SatScore', 'median'),
            ('SatScore',    'std')],
           )
3.4154302670623147


The difference between stay (churn = False) vs leave (churn = True) between our variables:

In [178]:
#single-example template:
#sat_avg_diff_pct = round(((satscore_df[('SatScore',   'mean')][1]) - (satscore_df[('SatScore',   'mean')][0]) ) *100,2) # (true-false)/false * 100

variables = [('SatScore',satscore_df,'points'),('CashbackAmount',cashback_df,'dollars'),('KmToWarehouse',distance_df,'km'),('DaysFromOrder',orderdays_df,'days'),('ActAgeMonths',tenure_stats,'months')]

def calc_avg_diff_pct(variable, df): 
    return round(((df[(variable,   'mean')][1]) - (df[(variable,   'mean')][0]) )/(df[(variable,   'mean')][0]) *100,2)

def calc_avg_diff(variable, df): 
    return round(((df[(variable,   'mean')][1]) - (df[(variable,   'mean')][0]) ),2)

avg_diff_list = []
abs_diff_list = []

for var in variables:
    var_name = var[0] 
    temp_df = var[1]
    avg_diff_list.append(calc_avg_diff_pct(var_name, temp_df))
    abs_diff_list.append(calc_avg_diff(var_name, temp_df))
    
for x in range(len(avg_diff_list)): 
    print("For {V}, the average percent difference between churned vs not churned accounts is {p}%".format(V = variables[x][0], p = avg_diff_list[x]))
    print("and the absolute difference is {d} {u} (Churn=yes perspective).".format(d=abs_diff_list[x], u = variables[x][2]))
    print("")



For SatScore, the average percent difference between churned vs not churned accounts is 13.06%
and the absolute difference is 0.39 points (Churn=yes perspective).

For CashbackAmount, the average percent difference between churned vs not churned accounts is -11.61%
and the absolute difference is -20.93 dollars (Churn=yes perspective).

For KmToWarehouse, the average percent difference between churned vs not churned accounts is 10.96%
and the absolute difference is 1.69 km (Churn=yes perspective).

For DaysFromOrder, the average percent difference between churned vs not churned accounts is -32.64%
and the absolute difference is -1.57 days (Churn=yes perspective).

For ActAgeMonths, the average percent difference between churned vs not churned accounts is -72.34%
and the absolute difference is -8.27 months (Churn=yes perspective).



**Numerical Summary:**

Surprisingly, the average churned customer has a nearly identical satisfaction score to the average non-churned customer. They received slightly fewer dollars back from the program in the month they cancelled their account - 11% or $20 less. They live slightly further from the warehouse than a non-churned customer, but this difference only amounts to 1.7km, which should not impact delivery times. They ordered, on average, 1.5 days more recently than their non-churned peers. 

For satisfaction, cashback, and distance, the differences between churned and non-churned customers is relatively small. The relative days from last order between the two groups is more significant (33% fewer for churned customers), however, the absolute difference of 1.5 days is negligible. 

The age of an account is a significant predictor of churn. A churned account is, on average, 72% newer or 8 months newer than a non-churned account. The average loyal customer has been with the company 11 months, while the average churned account is 3 months old. If customers remain with this service more than 3 months, they are likely to stay. 

**Recommendations:**

We recommend offering regular incentives to customers within the first three months of owning the account. Cash discounts totalling at least the $20 average difference between churned and loyal accounts.


Cleanup: now that we have these variables, we can empty the lists of dataframes: 

In [38]:
numeric_df_list.clear()

print(numeric_df_list)

[]


### Categorical Data Questions:

6. What is the % churn based on marital status category?
7. What is the % churn based on preferred order category

These we will look at slightly differently. For boolean data, because "True" = 1, the mean is the % of "True" responses, so, in our case, the % of churn for each category for each variable. ?

In [94]:
columns_cat = ['MaritalStatus','PreferedOrderCat']
cat_summary_methods = ['count','sum','mean'] #chosen summary methods for the .agg() method
cat_df_list = []

def categorical_col_analysis(column, sum_methods):
    temp_df = churn[[column,'Churn']]
    temp_df[column] = temp_df[column].astype('category')
    temp_df.dropna(inplace=True)
    temp_df_stats = temp_df.groupby(column).agg({'Churn':sum_methods})
    temp_df_stats['var_type'] = str(column)
    return temp_df_stats

for col in columns_cat: 
    cat_df_list.append(categorical_col_analysis(col, cat_summary_methods))

marital_df = cat_df_list[0]
order_df = cat_df_list[1]

print(marital_df)
print(order_df)
    

              Churn                      var_type
              count  sum      mean               
MaritalStatus                                    
Divorced        576   89  0.154514  MaritalStatus
Married        2055  230  0.111922  MaritalStatus
Single         1310  355  0.270992  MaritalStatus
                   Churn                         var_type
                   count  sum      mean                  
PreferedOrderCat                                         
Fashion              585   94  0.160684  PreferedOrderCat
Grocery              273   12  0.043956  PreferedOrderCat
Laptop & Accessory  1458  142  0.097394  PreferedOrderCat
Mobile               559  159  0.284436  PreferedOrderCat
Mobile Phone         887  255  0.287486  PreferedOrderCat
Others               179   12  0.067039  PreferedOrderCat


Experimenting with concatenating results: 

In [104]:
churn_categorical = pd.concat([order_df, marital_df])
print(churn_categorical)
print("~")
for x in churn_categorical.columns: print(x) #Just wanted to show multi-index nature of results

                   Churn                         var_type
                   count  sum      mean                  
Fashion              585   94  0.160684  PreferedOrderCat
Grocery              273   12  0.043956  PreferedOrderCat
Laptop & Accessory  1458  142  0.097394  PreferedOrderCat
Mobile               559  159  0.284436  PreferedOrderCat
Mobile Phone         887  255  0.287486  PreferedOrderCat
Others               179   12  0.067039  PreferedOrderCat
Divorced             576   89  0.154514     MaritalStatus
Married             2055  230  0.111922     MaritalStatus
Single              1310  355  0.270992     MaritalStatus
~
('Churn', 'count')
('Churn', 'sum')
('Churn', 'mean')
('var_type', '')


In [102]:
# Sanity check on mean = % for boolean: marital status, sum (total "True" churn) / count (total in category)
print(89/576, " - Divorced        576   89  0.154514")
print(230/2055, " - Married        2055  230  0.111922")
print(355/1310, " - Single         1310  355  0.270992")

0.1545138888888889  - Divorced        576   89  0.154514
0.11192214111922141  - Married        2055  230  0.111922
0.27099236641221375  - Single         1310  355  0.270992


In [194]:
# What % of our customers are currently married?
print(2055 / 3941)

0.5214412585638163


**Categorical Summary:**

Regarding preferred or most common account order category, the churn % for Grocery, Laptop & Accessory, and misc. other are relatively low, with grocery topping out at 96% loyalty. Fashion shows slightly higher churn at 16%, while the mobile and mobile phone departments show the highest churn at over 28% each. This could point to a disatisfaction with the mobile and phone product lines, or, the fact that these item categories are not everyday purchases, so customers sign up for an account long enough to buy what they need, then cancel. 

There is little difference between churn rate for divorced or married customers, but the churn rate for single customers is over 27%. It may be that our product is not appealing to this demographic, or, that this is just a reflection of the fact that single customers are more likely to be younger, with less room in their budget for discretionary spending.  

**Recommendations:**

We may decrease churn by cross-promoting our grocery and/or laptop accesory product lines beside mobile or phone products. It may be that some percent of our customers are unaware of the variety of our offerings. We could offer incentives in the form of discounts or promotional rates for grocery items to first-time mobile of phone customers. 

We could craft parallel targeted marketing campaigns, one to lean into our family-centric value to capitalize on our strength with the 52% of our customers which are married. The other could appeal to younger, unattached customers who may need a different story to embrace the value of staying with our company. 



##Project Discussion: Next Steps

If I were to continue with this project, I would look into: 

- Significance. There are other statistical functions in other libraries which can set up hypothetical tests to determine if our results differ significantly from what we would expect.

- Visualization. Before presenting results, it would be best to create graphs to make findings stand out. 